In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Stable Video Diffusion Img2Vid XT (Research Purpose)

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_img2vid_xt.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_img2vid_xt.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_img2vid_xt.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a>
    (a Python-3 GPU notebook with preinstalled HuggingFace/transformer libraries is recommended)
  </td>
</table>

## Overview

This notebook demonstrates running local inference for [stabilityai/stable-video-diffusion-img2vid-xt](https://huggingface.co/stabilityai/stable-video-diffusion-img2vid-xt) on either [Colab](https://colab.research.google.com) or [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench). This notebook also demonstrates deploying it on Vertex AI for batch prediction for research purpose only.

### Objective

- Run local predictions for image-to-video.
- Upload the model and run batch predictions for image-to-video.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

**NOTE**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

### Setup notebook

#### Colab only
Run the following commands for Colab and skip this section if you are using Workbench.

In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

! pip3 install --upgrade pip
! pip3 install torch==2.0.1+cu118  -f https://download.pytorch.org/whl/torch_stable.html
! pip3 install transformers==4.36.2
! pip3 install diffusers==0.25.0
! pip3 install datasets==2.9.0
! pip3 install accelerate==0.25.0

# Install gdown for downloading example training images.
! pip3 install gdown
# Remove wrong cublas version.
! pip3 uninstall nvidia_cublas_cu11 --yes

# Restart the notebook kernel after installs.
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

#### Workbench only
1.   Follow [this link](https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_2_1.ipynb) to deploy the notebook to a Vertex AI Workbench Instance.
2.   Select `Create a new Notebook`.
3.   Click `Advanced Options`.
4.   In the **Environment** tab, select `Debian 10` for **Operating System** and select `Custom Container` for **Environment**.
5.   Set `Docker container image` to `us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/transformers-notebook`.
6.   Under **Machine configuration**, select 1 `T4` GPU and select `Install NVIDIA GPU driver automatically for me`.
7.   Click `Create` to create the Vertex AI Workbench instance.

### Setup Google Cloud project

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs.

1. [Create a service account](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console) with `Vertex AI User` and `Storage Object Admin` roles for deploying fine tuned model to Vertex AI endpoint.

Fill following variables for experiments environment:

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = ""  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output. Fill it without the 'gs://' prefix.
GCS_BUCKET = ""  # @param {type:"string"}

# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

Initialize Vertex-AI API:

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

### Define constants

In [ ]:
# The pre-built serving docker image. It contains serving scripts and models.

SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-diffusers-serve-opt:20240223_1230_RC00"

### Define common functions

In [ ]:
import base64
import glob
import io
import os
from datetime import datetime
from io import BytesIO

import imageio
from diffusers.utils import load_image
from google.cloud import aiplatform, storage
from IPython.display import HTML


def load_and_resize_image(image_url):
    image = load_image(image_url)
    # required by the model
    new_width = round(image.size[0] / 8) * 8
    new_height = round(image.size[1] / 8) * 8
    return image.resize((new_width, new_height))


def create_job_name(prefix):
    user = os.environ.get("USER")
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    job_name = f"{prefix}-{user}-{now}"
    return job_name


def image_to_base64(image, format="JPEG"):
    buffer = BytesIO()
    image.save(buffer, format=format)
    image_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return image_str


def display_video_frames(frames, fps=7, width=500):
    io_obj = io.BytesIO()
    imageio.mimsave(io_obj, frames, format=".mp4", fps=fps)
    video_data = base64.b64encode(io_obj.getvalue()).decode("utf-8")
    html = ""
    html += f"<video controls width={width}>"
    html += f'<source src="data:video/mp4;base64,{video_data}" type="video/mp4">'
    html += "</video>"
    display(HTML(html))


def upload_model(model_id, task):
    model_name = model_id
    serving_env = {
        "MODEL_ID": model_id,
        "TASK": task,
    }
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/diffusers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    return model


def get_bucket_and_blob_name(filepath):
    # The gcs path is of the form gs://<bucket-name>/<blob-name>
    gs_suffix = filepath.split("gs://", 1)[1]
    return tuple(gs_suffix.split("/", 1))


def upload_local_dir_to_gcs(local_dir_path, gcs_dir_path):
    """Uploads files in a local directory to a GCS directory."""
    client = storage.Client()
    bucket_name = gcs_dir_path.split("/")[2]
    bucket = client.get_bucket(bucket_name)
    for local_file in glob.glob(local_dir_path + "/**"):
        if not os.path.isfile(local_file):
            continue
        filename = local_file[1 + len(local_dir_path) :]
        gcs_file_path = os.path.join(gcs_dir_path, filename)
        _, blob_name = get_bucket_and_blob_name(gcs_file_path)
        blob = bucket.blob(blob_name)
        blob.upload_from_filename(local_file)
        print("Copied {} to {}.".format(local_file, gcs_file_path))


def download_gcs_blob_as_json(gcs_file_path):
    """Download GCS blob and convert it to json."""
    client = storage.Client()
    bucket_name, blob_name = get_bucket_and_blob_name(gcs_file_path)
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    return json.loads(blob.download_as_bytes())

## Image to Video

In [ ]:
# @title Define model and task
model_id = "stabilityai/stable-video-diffusion-img2vid-xt"
task = "image-to-video"

In [ ]:
# @title Run inferences locally
import torch
from diffusers import StableVideoDiffusionPipeline

pipe = StableVideoDiffusionPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16, variant="fp16"
)
pipe.enable_model_cpu_offload()

# Load the conditioning image
image = load_and_resize_image(
    "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/svd/rocket.png"
)

generator = torch.manual_seed(42)
frames_list = pipe([image], decode_chunk_size=8, generator=generator).frames

display_video_frames(frames_list[0], fps=7)

### Deploy model to run prediction

Deploy the stable video diffusion model for the `image-to-video` task.

Once deployed, you can run batch prediction with a batch of images to generate videos.

When deployed on one `NVIDIA_L4` GPU, the averaged inference time of a request is ~600 seconds.

In [ ]:
# @title Define upload model

model = upload_model(model_id=model_id, task=task)

In [ ]:
# @title Prepare input
import json

image = load_and_resize_image(
    "https://images.pexels.com/videos/853848/free-video-853848.jpg"
)
display(image)

instances = [
    {"prompt": "placehoder", "image": image_to_base64(image), "motion_bucket_id": 127}
]

# Convert and write JSON object to file
os.makedirs("bath_prediction_input", exist_ok=True)
with open("bath_prediction_input/input.jsonl", "w") as outfile:
    for item in instances:
        json_str = json.dumps(item)
        outfile.write(json_str)
        outfile.write("\n")

upload_local_dir_to_gcs(
    "bath_prediction_input", f"gs://{GCS_BUCKET}/image_to_video/input"
)

Upload model. For batch predictions, only model upload is needed. you don't need to deploy the model to any endpoint.

In [ ]:
# @title Run batch prediction

machine_type = "n1-standard-32"
accelerator_type = "NVIDIA_TESLA_P100"
accelerator_count = 2

batch_prediction_job = model.batch_predict(
    instances_format="jsonl",
    job_display_name="batch_predict_image_to_video",
    gcs_source=[f"gs://{GCS_BUCKET}/image_to_video/input/input.jsonl"],
    gcs_destination_prefix=f"gs://{GCS_BUCKET}/image_to_video/output",
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    starting_replica_count=1,
)

batch_prediction_job.wait()

print(batch_prediction_job.display_name)
print(batch_prediction_job.resource_name)
print(batch_prediction_job.state)

In [ ]:
# @title Display results

gcs_output_file = (
    batch_prediction_job.output_info.gcs_output_directory
    + "/prediction.results-00000-of-00001"
)
prediction_result = download_gcs_blob_as_json(gcs_output_file)


html = ""
html += "<video controls width=500>"
html += f'<source src="data:video/mp4;base64,{prediction_result["prediction"]}" type="video/mp4">'
html += "</video>"
HTML(html)

In [ ]:
# @title Delete model.
model.delete()